In [1]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

"""
parser = argparse.ArgumentParser(description='VAE MNIST Example')
parser.add_argument('--batch-size', type=int, default=128, metavar='N',
                    help='input batch size for training (default: 128)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='enables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
args = parser.parse_args()
args.cuda = not args.no_cuda and torch.cuda.is_available()
"""


cuda = True
batch_size = 128
epochs = 10
seed = 1
log_interval = 10




#torch.manual_seed(args.seed)
torch.manual_seed(seed)

#device = torch.device("cuda" if args.cuda else "cpu")
device = torch.device("cuda" if cuda else "cpu")

#kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)


class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        #print(logvar.shape)
        if self.training:
            std = torch.exp(0.5*logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return F.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)


# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    # recon_x is output from the activation layer. if recon_x has not gone through sigmoid activation, 
    # use binary_cross_entropy_with_logits
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), size_average=False)
    print("BCE: "+str(BCE/batch_size))

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    #tmp = 1 + logvar - mu.pow(2) - logvar.exp()
    #print("tmp: "+str(tmp.shape))
    #KLD = -0.5*torch.sum(tmp)
    #print("KLD: "+str(KLD.shape))
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    #print(BCE)

    return BCE + KLD


def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        #print("recon: "+str(recon_batch.shape))
        #print("var: "+str(torch.exp(logvar)))
        loss = loss_function(recon_batch, data, mu, logvar)
        #print("loss: "+str(loss.item()/len(data)))
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        #print(len(data))
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))
            #print("loss "+str(loss.item()))
            #print("len "+str(len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))


for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        sample = torch.randn(64, 20).to(device)
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, 1, 28, 28),
                   'results/sample_' + str(epoch) + '.png')

BCE: tensor(550.4553, device='cuda:0')
Train Epoch: 1 [0/60000 (0%)]	Loss: 550.533936
BCE: tensor(524.4629, device='cuda:0')
BCE: tensor(498.4903, device='cuda:0')
BCE: tensor(472.5098, device='cuda:0')
BCE: tensor(445.9448, device='cuda:0')
BCE: tensor(413.1568, device='cuda:0')
BCE: tensor(376.8734, device='cuda:0')
BCE: tensor(342.9244, device='cuda:0')
BCE: tensor(307.4867, device='cuda:0')
BCE: tensor(286.3045, device='cuda:0')
BCE: tensor(271.4376, device='cuda:0')
Train Epoch: 1 [1280/60000 (2%)]	Loss: 303.023163
BCE: tensor(269.9990, device='cuda:0')
BCE: tensor(242.1886, device='cuda:0')
BCE: tensor(246.1439, device='cuda:0')
BCE: tensor(234.0586, device='cuda:0')
BCE: tensor(232.4550, device='cuda:0')
BCE: tensor(238.2680, device='cuda:0')
BCE: tensor(232.4213, device='cuda:0')
BCE: tensor(234.8048, device='cuda:0')
BCE: tensor(235.3544, device='cuda:0')
BCE: tensor(228.2004, device='cuda:0')
Train Epoch: 1 [2560/60000 (4%)]	Loss: 235.810791
BCE: tensor(228.2328, device='cuda

BCE: tensor(140.3069, device='cuda:0')
BCE: tensor(142.1251, device='cuda:0')
BCE: tensor(140.8108, device='cuda:0')
BCE: tensor(143.3246, device='cuda:0')
BCE: tensor(134.8968, device='cuda:0')
BCE: tensor(135.5101, device='cuda:0')
Train Epoch: 1 [25600/60000 (43%)]	Loss: 150.882858
BCE: tensor(135.9858, device='cuda:0')
BCE: tensor(132.6208, device='cuda:0')
BCE: tensor(135.2714, device='cuda:0')
BCE: tensor(139.0252, device='cuda:0')
BCE: tensor(145.5138, device='cuda:0')
BCE: tensor(140.6665, device='cuda:0')
BCE: tensor(134.7077, device='cuda:0')
BCE: tensor(138.4087, device='cuda:0')
BCE: tensor(147.1949, device='cuda:0')
BCE: tensor(138.5897, device='cuda:0')
Train Epoch: 1 [26880/60000 (45%)]	Loss: 154.950745
BCE: tensor(135.0194, device='cuda:0')
BCE: tensor(133.8106, device='cuda:0')
BCE: tensor(133.7702, device='cuda:0')
BCE: tensor(139.9744, device='cuda:0')
BCE: tensor(137.2571, device='cuda:0')
BCE: tensor(135.7049, device='cuda:0')
BCE: tensor(141.2299, device='cuda:0')

BCE: tensor(115.5907, device='cuda:0')
Train Epoch: 1 [49920/60000 (83%)]	Loss: 136.630585
BCE: tensor(112.5023, device='cuda:0')
BCE: tensor(115.8947, device='cuda:0')
BCE: tensor(116.9788, device='cuda:0')
BCE: tensor(109.1163, device='cuda:0')
BCE: tensor(115.9317, device='cuda:0')
BCE: tensor(110.4351, device='cuda:0')
BCE: tensor(111.7496, device='cuda:0')
BCE: tensor(113.4461, device='cuda:0')
BCE: tensor(110.6179, device='cuda:0')
BCE: tensor(118.0482, device='cuda:0')
Train Epoch: 1 [51200/60000 (85%)]	Loss: 138.097305
BCE: tensor(115.3419, device='cuda:0')
BCE: tensor(112.3787, device='cuda:0')
BCE: tensor(113.3605, device='cuda:0')
BCE: tensor(112.4492, device='cuda:0')
BCE: tensor(117.6300, device='cuda:0')
BCE: tensor(109.3702, device='cuda:0')
BCE: tensor(113.3417, device='cuda:0')
BCE: tensor(114.3627, device='cuda:0')
BCE: tensor(112.7672, device='cuda:0')
BCE: tensor(113.9465, device='cuda:0')
Train Epoch: 1 [52480/60000 (87%)]	Loss: 132.911041
BCE: tensor(108.7418, dev

BCE: tensor(104.9141, device='cuda:0')
Train Epoch: 2 [6400/60000 (11%)]	Loss: 125.809158
BCE: tensor(103.5047, device='cuda:0')
BCE: tensor(106.3395, device='cuda:0')
BCE: tensor(105.0266, device='cuda:0')
BCE: tensor(105.2963, device='cuda:0')
BCE: tensor(105.6754, device='cuda:0')
BCE: tensor(102.4691, device='cuda:0')
BCE: tensor(104.3124, device='cuda:0')
BCE: tensor(105.7452, device='cuda:0')
BCE: tensor(102.4893, device='cuda:0')
BCE: tensor(107.9761, device='cuda:0')
Train Epoch: 2 [7680/60000 (13%)]	Loss: 129.146851
BCE: tensor(104.8931, device='cuda:0')
BCE: tensor(101.4401, device='cuda:0')
BCE: tensor(102.4455, device='cuda:0')
BCE: tensor(108.1486, device='cuda:0')
BCE: tensor(103.3183, device='cuda:0')
BCE: tensor(104.4914, device='cuda:0')
BCE: tensor(104.4707, device='cuda:0')
BCE: tensor(110.8865, device='cuda:0')
BCE: tensor(106.8913, device='cuda:0')
BCE: tensor(106.1307, device='cuda:0')
Train Epoch: 2 [8960/60000 (15%)]	Loss: 127.163307
BCE: tensor(108.9270, device

BCE: tensor(100.2165, device='cuda:0')
Train Epoch: 2 [30720/60000 (51%)]	Loss: 122.977409
BCE: tensor(99.4167, device='cuda:0')
BCE: tensor(102.0073, device='cuda:0')
BCE: tensor(98.4584, device='cuda:0')
BCE: tensor(98.2431, device='cuda:0')
BCE: tensor(103.3540, device='cuda:0')
BCE: tensor(99.1083, device='cuda:0')
BCE: tensor(95.8166, device='cuda:0')
BCE: tensor(98.6326, device='cuda:0')
BCE: tensor(97.8230, device='cuda:0')
BCE: tensor(97.7437, device='cuda:0')
Train Epoch: 2 [32000/60000 (53%)]	Loss: 120.314651
BCE: tensor(96.8327, device='cuda:0')
BCE: tensor(99.7467, device='cuda:0')
BCE: tensor(95.0701, device='cuda:0')
BCE: tensor(104.2478, device='cuda:0')
BCE: tensor(103.8524, device='cuda:0')
BCE: tensor(95.3147, device='cuda:0')
BCE: tensor(98.1987, device='cuda:0')
BCE: tensor(101.3011, device='cuda:0')
BCE: tensor(98.4934, device='cuda:0')
BCE: tensor(93.7872, device='cuda:0')
Train Epoch: 2 [33280/60000 (55%)]	Loss: 115.683502
BCE: tensor(96.7326, device='cuda:0')
BC

...

BCE: tensor(80.3407, device='cuda:0')
BCE: tensor(84.6497, device='cuda:0')
BCE: tensor(80.0138, device='cuda:0')
BCE: tensor(84.2781, device='cuda:0')
BCE: tensor(83.0990, device='cuda:0')
BCE: tensor(82.8100, device='cuda:0')
BCE: tensor(80.7626, device='cuda:0')
Train Epoch: 9 [29440/60000 (49%)]	Loss: 106.098381
BCE: tensor(76.6055, device='cuda:0')
BCE: tensor(82.0713, device='cuda:0')
BCE: tensor(82.6287, device='cuda:0')
BCE: tensor(83.0821, device='cuda:0')
BCE: tensor(86.6925, device='cuda:0')
BCE: tensor(79.9447, device='cuda:0')
BCE: tensor(83.2963, device='cuda:0')
BCE: tensor(81.3017, device='cuda:0')
BCE: tensor(77.9565, device='cuda:0')
BCE: tensor(80.1601, device='cuda:0')
Train Epoch: 9 [30720/60000 (51%)]	Loss: 104.752144
BCE: tensor(79.8599, device='cuda:0')
BCE: tensor(80.6151, device='cuda:0')
BCE: tensor(81.2475, device='cuda:0')
BCE: tensor(81.7957, device='cuda:0')
BCE: tensor(79.3487, device='cuda:0')
BCE: tensor(80.6032, device='cuda:0')
BCE: tensor(83.5897, d

BCE: tensor(82.1261, device='cuda:0')
BCE: tensor(83.4655, device='cuda:0')
BCE: tensor(82.1524, device='cuda:0')
Train Epoch: 9 [53760/60000 (90%)]	Loss: 107.741348
BCE: tensor(79.8550, device='cuda:0')
BCE: tensor(77.4068, device='cuda:0')
BCE: tensor(84.6033, device='cuda:0')
BCE: tensor(82.9290, device='cuda:0')
BCE: tensor(81.7094, device='cuda:0')
BCE: tensor(83.6018, device='cuda:0')
BCE: tensor(81.5666, device='cuda:0')
BCE: tensor(79.9140, device='cuda:0')
BCE: tensor(80.5347, device='cuda:0')
BCE: tensor(82.3408, device='cuda:0')
Train Epoch: 9 [55040/60000 (92%)]	Loss: 108.215126
BCE: tensor(81.8898, device='cuda:0')
BCE: tensor(80.9087, device='cuda:0')
BCE: tensor(81.3560, device='cuda:0')
BCE: tensor(82.4159, device='cuda:0')
BCE: tensor(81.8942, device='cuda:0')
BCE: tensor(84.4698, device='cuda:0')
BCE: tensor(76.9810, device='cuda:0')
BCE: tensor(82.7029, device='cuda:0')
BCE: tensor(82.7050, device='cuda:0')
BCE: tensor(82.0561, device='cuda:0')
Train Epoch: 9 [56320/

BCE: tensor(79.4468, device='cuda:0')
BCE: tensor(75.5564, device='cuda:0')
BCE: tensor(81.0668, device='cuda:0')
BCE: tensor(83.3612, device='cuda:0')
BCE: tensor(81.7592, device='cuda:0')
BCE: tensor(86.6559, device='cuda:0')
Train Epoch: 10 [10240/60000 (17%)]	Loss: 111.903900
BCE: tensor(80.4227, device='cuda:0')
BCE: tensor(82.9741, device='cuda:0')
BCE: tensor(80.8957, device='cuda:0')
BCE: tensor(81.2642, device='cuda:0')
BCE: tensor(81.9029, device='cuda:0')
BCE: tensor(81.0512, device='cuda:0')
BCE: tensor(82.6356, device='cuda:0')
BCE: tensor(80.5613, device='cuda:0')
BCE: tensor(78.3603, device='cuda:0')
BCE: tensor(79.7966, device='cuda:0')
Train Epoch: 10 [11520/60000 (19%)]	Loss: 104.422806
BCE: tensor(80.4089, device='cuda:0')
BCE: tensor(80.1804, device='cuda:0')
BCE: tensor(80.0642, device='cuda:0')
BCE: tensor(86.0708, device='cuda:0')
BCE: tensor(82.1227, device='cuda:0')
BCE: tensor(80.0772, device='cuda:0')
BCE: tensor(80.6121, device='cuda:0')
BCE: tensor(79.7581,

BCE: tensor(77.7049, device='cuda:0')
BCE: tensor(80.5116, device='cuda:0')
Train Epoch: 10 [34560/60000 (58%)]	Loss: 106.224648
BCE: tensor(81.2141, device='cuda:0')
BCE: tensor(82.5149, device='cuda:0')
BCE: tensor(83.1400, device='cuda:0')
BCE: tensor(80.3127, device='cuda:0')
BCE: tensor(80.8088, device='cuda:0')
BCE: tensor(78.5407, device='cuda:0')
BCE: tensor(86.4324, device='cuda:0')
BCE: tensor(79.6420, device='cuda:0')
BCE: tensor(82.6584, device='cuda:0')
BCE: tensor(81.6568, device='cuda:0')
Train Epoch: 10 [35840/60000 (60%)]	Loss: 107.885605
BCE: tensor(79.4548, device='cuda:0')
BCE: tensor(79.5058, device='cuda:0')
BCE: tensor(80.3042, device='cuda:0')
BCE: tensor(77.1753, device='cuda:0')
BCE: tensor(84.4442, device='cuda:0')
BCE: tensor(81.9859, device='cuda:0')
BCE: tensor(81.7222, device='cuda:0')
BCE: tensor(84.3543, device='cuda:0')
BCE: tensor(80.4401, device='cuda:0')
BCE: tensor(83.5729, device='cuda:0')
Train Epoch: 10 [37120/60000 (62%)]	Loss: 109.065811
BCE: 

BCE: tensor(76.8485, device='cuda:0')
BCE: tensor(80.0884, device='cuda:0')
BCE: tensor(82.2178, device='cuda:0')
BCE: tensor(79.9798, device='cuda:0')
BCE: tensor(78.4674, device='cuda:0')
BCE: tensor(80.8303, device='cuda:0')
BCE: tensor(85.3770, device='cuda:0')
BCE: tensor(61.9431, device='cuda:0')
====> Epoch: 10 Average loss: 106.3760
BCE: tensor(71.9850, device='cuda:0')
BCE: tensor(71.5086, device='cuda:0')
BCE: tensor(71.9345, device='cuda:0')
BCE: tensor(74.8455, device='cuda:0')
BCE: tensor(72.2213, device='cuda:0')
BCE: tensor(73.1502, device='cuda:0')
BCE: tensor(73.8238, device='cuda:0')
BCE: tensor(69.2600, device='cuda:0')
BCE: tensor(71.3151, device='cuda:0')
BCE: tensor(71.1923, device='cuda:0')
BCE: tensor(73.4261, device='cuda:0')
BCE: tensor(73.1665, device='cuda:0')
BCE: tensor(69.8356, device='cuda:0')
BCE: tensor(70.2552, device='cuda:0')
BCE: tensor(70.4265, device='cuda:0')
BCE: tensor(72.3901, device='cuda:0')
BCE: tensor(71.5564, device='cuda:0')
BCE: tensor

In [12]:
x = torch.tensor([[1,2],[3,4]])
t = torch.tensor([[1,1],[1,1]])
x = (x>t).float()
x

tensor([[ 0.,  1.],
        [ 1.,  1.]])

In [ ]:
train_loader

In [5]:
for batch_idx, (data, _) in enumerate(train_loader):
    print(type(data))
    print(data.shape)
    #d = data.view(-1, 784)
    #print (d.shape)

<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torc

<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torc

<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([128, 1, 28, 28])
<class 'torc

In [5]:
x = torch.randn(2,4)
x

tensor([[-0.3111, -0.9662, -0.7928,  0.6137],
        [-0.1155,  0.2167,  0.5646, -3.1376]])

In [7]:
x.view(8)

tensor([-0.3111, -0.9662, -0.7928,  0.6137, -0.1155,  0.2167,  0.5646,
        -3.1376])